In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn #neural network module
import torch.optim as optim #optimizer
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import time as time
from tqdm import tqdm

import pickle
import gc
import os

from Preprocessing import *

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU available')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print('mps available')
else:
    device = torch.device("cpu")
    print('GPU not available')

mps available


### Dataset and Dataloader

In [3]:
datasetpath=os.path.realpath('Dataset_CP.pt')
PolDataset = os.path.realpath('PolyphonicDataset.pt')

#Loading monophonic and polyphonic classes
class MonophonicDataset(Dataset):

   def __init__(self, Instruments):

      DS = torch.load(datasetpath)
      self.Data = []
      self.Instruments = Instruments

      for inst in Instruments:

        RandomData = np.random.choice(DS[inst], int(len(DS[inst])*0.7))
        self.Data.extend(RandomData)

      del DS
      gc.collect()


   def __len__(self):
      return len(self.Data)

   def __getitem__(self, idx):

      PreviousBars = self.Data[idx]['Bars'][0].to_dense()
      Bars = self.Data[idx]['Bars'][1].to_dense()

      prog = self.Data[idx]['Program']
      tempo = self.Data[idx]['Tempo'][0]

      Cond1D = torch.tensor([tempo, prog], dtype=torch.int, device=Bars.device)
      return Bars, PreviousBars, Cond1D




class PolyphonicDataset(Dataset):

   def __init__(self, Genre, EightBars = False):

         DS = torch.load(PolDataset, weights_only=False)
         self.Data = []
         self.Genre = Genre
         self.EightBars = EightBars

         for gen in Genre:
            self.Data.extend(DS[gen])

         if self.EightBars:
            self.EightDataset = EightBarsDataset(self.Data, Mono = False)

         del DS
         gc.collect()

   def __len__(self):

      if self.EightBars:
         return len(self.EightDataset)
      else:
         return len(self.Data)

   def __getitem__(self, idx):

      if not self.EightBars:

         PreviousBars = self.Data[idx]['Bars'][0].to_dense()
         Bars = self.Data[idx]['Bars'][1].to_dense()

         prog = self.Data[idx]['Program'][0]
         tempo = self.Data[idx]['Tempo'][0]

         TEMPO_MIN, TEMPO_MAX = 60, 200
         PROGRAM_MIN, PROGRAM_MAX = 1, 128

         tempo_norm = (tempo - TEMPO_MIN) / (TEMPO_MAX - TEMPO_MIN)
         prog_norm = [(p - PROGRAM_MIN) / (PROGRAM_MAX - PROGRAM_MIN) for p in prog]

         Cond1D = torch.tensor([tempo_norm] + prog_norm, dtype=torch.float, device=Bars.device)
         return Bars, PreviousBars, Cond1D
      
      else:         
         Bars = self.EightDataset[idx]['Bars'].to_dense()
         return Bars


### Concatenation

Definition of the concatenation functions that are used in the hidden layers to concatenate the output and the 1_d and 2_d conditions.

1_d conditioning vector of shape $[n,1]$ with an output of shape $[batch,features,a,b]$:
* first we have to duplicate the vector $a\cdot b$ times to get a tensor of shape $[batch,n,a,b]$
* then we can concatenate the two tensors in the depth dimension (i.e dim=1)

2_d conditioning matrix of the same shape of the output $[batch,features,a,b]$ except the depth dim (it must be that because how we build the conditioner CNN):
* first we check that the dimensions are correct
* we concatenate the two tensors in the depth dimension (i.e dim=1)

In [4]:
def conv_cond_concat(x, y):
    """Concatenate conditioning vector on feature map axis."""
    x_shapes = x.shape  #[batch,n_features,a,b]
    y_shapes = y.shape  #[batch,n]
    y2 = y.view(x_shapes[0],y_shapes[1],1,1)                              #[batch,n,1,1]
    y2 = y2.expand(x_shapes[0],y_shapes[1],x_shapes[2],x_shapes[3])      #[batch,n,a,b]

    return torch.cat((x, y2),dim=1)                                     #[batch,n_features+n,a,b]

### The Generator and the Conditioner

The generator uses `ConvTranspose2d` (upsampling) layers to produce an image from a seed (random noise). Start with two `Dense` layers that take this seed as input and transform it to a tensor of shape $[batch size,]$, then upsample several times until we reach the desired size of a bar of $[instrument,128,16]$. We use  the `ReLU` activation for each layer, except the output layer which can use `Sigmoid` to predict pixel values in the [0, 1] range.

Coupled to the generator there is the conditioner that uses `Conv2d` (sampling) layers to produce the 2_d tensors that serve as informations from the preaviou bar. The conditioner can be viewed as the reverse of the generator because it uses filters with the same shapes of the ones in the generator. In this case we use  the `LeakyReLU` activation for each layer.

In [5]:
class Generator(nn.Module):

    def __init__(self, input_size, cond_1d_size, instrument_size=1, n_hlayers=128):
            super().__init__()

            self.input_size = input_size
            self.instrument_size = instrument_size
            self.cond1d_dim = cond_1d_size
            self.nhlayers=n_hlayers

            #generator layers
            #as said in the DCGAN paper always ReLU ctivation function in the generator excluded the last layer
            #as said in the DCGAN paper always batchnorm in the generator excluded the last layer
            self.ff1 = nn.Sequential(
                nn.Linear(input_size+cond_1d_size, n_hlayers*5*5),                                                                            #[batch,n_hlayers*5*5]
                nn.BatchNorm1d(n_hlayers*5*5),
                nn.ReLU(inplace=False)
                )

            #reshape to [batch size,128,5,5]
            #+condition [batch,128+cond_1d_size,5,5]
            self.cnn1 = nn.Sequential(
                nn.ConvTranspose2d(n_hlayers+cond_1d_size, 128, kernel_size=3, stride=1, bias=False, padding=0),           #[batch,128,7,7]
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=False)
                )
            #+condition [batch,128+cond_1d_size,13,9]
            self.cnn2 = nn.Sequential(
                nn.ConvTranspose2d(128+cond_1d_size, 64, kernel_size=3, stride=2, bias=False, padding=1),           #[batch,64,13,13]
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=False)
                )
            #+condition [batch,64+cond_1d_size,13,13]
            self.cnn3h = nn.Sequential(
                nn.ConvTranspose2d(64+cond_1d_size, 64, kernel_size=(1,7), stride=(1,3), bias=False, padding=(0,1), output_padding=(0,1)),           #[batch,64,42,13]
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=False)
                )
            #+condition [batch,64+cond_1d_size,42,13]
            self.cnn3v = nn.Sequential(
                nn.ConvTranspose2d(64+cond_1d_size, 64, kernel_size=(7,1), stride=(3,1), bias=False, padding=(1,0), output_padding=(1,0)),           #[batch,64,42,42]
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=False)
                )
            #+condition [batch,64+cond_1d_size,42,42]
            self.cnn4 = nn.Sequential(
                nn.ConvTranspose2d(64+cond_1d_size, instrument_size, kernel_size=5, stride=3, bias=False, padding=0),       #[batch,instrument_size,128,128]
                nn.Sigmoid()
                #Sigmoid funciotn because we want to generate the matrixes of music without velocity, i.e. only (0,1)
                #Thus we use the sigmoid which is a smoother version of the sign function
                )



    def forward(self, z, cond_1d, batch_size):



            #single cond_1d size =[n,1], batch_cond_1d size = [batch_size,n]

            input = torch.cat((z,cond_1d), dim=1)   #[batch_size, input_size+cond_1d_size]

            h1 = self.ff1(input)                    #[batch,3200]
            h1 = h1.reshape(batch_size, self.nhlayers, 5, 5)  #[batch,128,5,5]
            h1 = conv_cond_concat(h1,cond_1d)       #[batch,128+cond_1d_size,5,5]


            h2 = self.cnn1(h1)                      #[batch,128,7,7]
            h2 = conv_cond_concat(h2,cond_1d)       #[batch,128+cond_1d_size,7,7]


            h3 = self.cnn2(h2)                      #[batch,64,13,13]
            h3 = conv_cond_concat(h3,cond_1d)       #[batch,64+cond_1d_size,13,13]


            h4 = self.cnn3h(h3)                     #[batch,64,42,13]
            h4 = conv_cond_concat(h4,cond_1d)       #[batch,64+cond_1d_size,42,13]

            h5 = self.cnn3v(h4)                     #[batch,64,42,42]
            h5 = conv_cond_concat(h5,cond_1d)       #[batch,64+cond_1d_size,42,42]

            out = self.cnn4(h5)                     #[batch,instrument_size,128,128]

            return out

### The Discriminator

The discriminator uses `Conv2d` (sampling) layers to produce a scalar output from a bar input. Start with two `Conv2d` layers that reduce the size of the input, then use two `Dense` layers. We use  the `LeakyReLU` activation for each layer, except the output layer which can use `Sigmoid` to predict true-false probability value in the [0, 1] range. Note that the activation is included in the loss function.

In [6]:
class Discriminator(nn.Module):

    def __init__(self, cond_1d_size, instrument_size=1):
        super().__init__()

        self.instrument_size = instrument_size
        self.cond1d_dim = cond_1d_size

        #as said in the DCGAN paper always batchnorm in the discriminator layers excluded the first layer
        self.cnn1 = nn.Sequential(
            nn.Conv2d(instrument_size+cond_1d_size, 32, kernel_size=5, stride=3, padding=0),        #[batch,32,42,42]
            nn.LeakyReLU(inplace=False)
        )
        #+condition [batch,64+cond_1d_size,42,12]
        self.cnn2 = nn.Sequential(
            nn.Conv2d(32+cond_1d_size, 64, kernel_size=5, stride=3, padding=1),                       #[batch,64,13,13]
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=False)
        )
        #+condition [batch,64+cond_1d_size,19,8]
        self.cnn3 = nn.Sequential(
            nn.Conv2d(64+cond_1d_size, 128, kernel_size=3, stride=2, padding=1),                             #[batch,128,7,7]
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=False)
        )
        #+condition [batch,64+cond_1d_size,19,8]
        self.cnn4 = nn.Sequential(
            nn.Conv2d(128+cond_1d_size, 128, kernel_size=3, stride=2, padding=0),                             #[batch,128,3,3]
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=False)
        )

        #+condition [batch,1152+cond_1d_size]
        self.ffnn1 = nn.Linear(1152+cond_1d_size, 1)      #no sigmoid activation function because it is already in the definition of the cross entropy loss function





    def forward(self, x, cond_1d):
        input = conv_cond_concat(x,cond_1d)         #[batch,instrument_size+cond_1d_size,128,128]

        h0 = self.cnn1(input)                       #[batch,32,42,42]
        fm=h0
        h0 = conv_cond_concat(h0, cond_1d)          #[batch,32+cond_1d_size,42,42]

        h1 = self.cnn2(h0)                          #[batch,64,13,13]
        h1 = conv_cond_concat(h1,cond_1d)           #[batch,64+cond_1d_size,13,13]

        h2 = self.cnn3(h1)                          #[batch,128,7,7]
        h2 = conv_cond_concat(h2,cond_1d)           #[batch,128+cond_1d_size,7,7]

        h3 = self.cnn4(h2)                          #[batch,128,3,3]
        h3 = torch.flatten(h3, 1)                   #[batch,1152]
        h3 = torch.cat((h3,cond_1d),dim=1)          #[batch,1152+cond_1d_size]

        h4 = self.ffnn1(h3)                         #[batch,1]
        h4_sigmoid = torch.sigmoid(h4)


        return h4_sigmoid, h4, fm



### Weights initialization

Is this ok?

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.xavier_uniform_(m.weight.data)
    if classname.find('Linear') != -1:
        nn.init.xavier_uniform_(m.weight.data)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.2)
        nn.init.constant_(m.bias.data, 0)

### Creation of the model

In [8]:
generator = Generator(input_size=100, cond_1d_size=5, instrument_size=4, n_hlayers=128)
generator.apply(weights_init)

Generator(
  (ff1): Sequential(
    (0): Linear(in_features=105, out_features=3200, bias=True)
    (1): BatchNorm1d(3200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn1): Sequential(
    (0): ConvTranspose2d(133, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn2): Sequential(
    (0): ConvTranspose2d(133, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn3h): Sequential(
    (0): ConvTranspose2d(69, 64, kernel_size=(1, 7), stride=(1, 3), padding=(0, 1), output_padding=(0, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (cnn3v): Sequential(
    (0): ConvTranspose2d(69, 64, kernel_size=(7, 1), stride=(3, 1), padding=(1

In [9]:
discriminator = Discriminator(cond_1d_size=5, instrument_size=4)
discriminator.apply(weights_init)

Discriminator(
  (cnn1): Sequential(
    (0): Conv2d(9, 32, kernel_size=(5, 5), stride=(3, 3))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (cnn2): Sequential(
    (0): Conv2d(37, 64, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (cnn3): Sequential(
    (0): Conv2d(69, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (cnn4): Sequential(
    (0): Conv2d(133, 128, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
  )
  (ffnn1): Linear(in_features=1157, out_features=1, bias=True)
)

### Dimension testing

Produce a noise vector of size `[10, 100]`, a noise 1d condition vector of size `[10, 15]`, and a noise 2d condition tensor of size `[10, 1, 128,16]`. Note that we need a 1d and a 2d contions for each batch input. Then we use the (as yet **untrained**) generator to create an image of expected output shape $[10,1,128,16]$.

Then use the (yet **untrained**) discriminator to classify the generated images as real or fake. The model will be trained to output the probability that the image is real in the first output component, thus we expect an output vector of size `[10, 1]` with $x_i \in [0,1]$.

In [10]:
# ############################ input (batch_size=10, vector_size=100) ###############################
# noise = torch.normal(0, 1, [50, 100])
# print(noise.shape)
# ############################ conditions ###############################
# cond_1d =  torch.normal(0,1,[50,2])
# ############################ generator ###############################
# generated_bar = generator(noise, cond_1d, batch_size=50).detach()
# print(generated_bar.shape)
# ############################ discriminator ###############################
# decision, __, __= discriminator(generated_bar, cond_1d)
# print(decision)

### Discriminator loss

This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.
The discriminator loss is of the form:

$\frac{1}{m}\sum_{i=1}^{m}-[\log D(\boldsymbol{x}^{(i)}) +\log(1-D(G(\boldsymbol{z}^{(i)})))]$

We inplement one-sided label smoothing to penalize self confidence and imporve the convergence of the training. Thus we substitute the discriminator's predictions on real images to an array of 1s with an array of (1-$\alpha$)s and the loss function becomes:

$\frac{1}{m}\sum_{i=1}^{m}-[(1-\alpha) \log D(\boldsymbol{x}^{(i)}) +\alpha \log (1-D(\boldsymbol{x}^{(i)}))+\log(1-D(G(\boldsymbol{z}^{(i)})))]$

In [11]:
cross_entropy = nn.BCEWithLogitsLoss()
MSE=nn.MSELoss()

In [12]:
def discriminator_loss(real_output, fake_output, device, alpha=0.1):

    #real_targets = torch.ones_like(real_output, device=device)                 #no label smoothing -> True output expected output is 1
    real_targets = torch.full_like(real_output, 1.0 - alpha, device=device)     #one side label smoothing to penalize self confidence
    fake_targets = torch.zeros_like(fake_output, device=device)                 #no label smoothing -> Fake output expected output is 0

    real_loss = cross_entropy(real_output, real_targets)
    fake_loss = cross_entropy(fake_output, fake_targets)

    total_loss = real_loss + fake_loss
    return total_loss

### Generator loss

The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1).
The generator loss is of the form:

$\frac{1}{m}\sum_{i=1}^{m}\log(1-D(G(\boldsymbol{z}^{(i)})))$

However this loss has some convergence issues due to vanishing gradients. So instead we use the following loss which has the same trend but stronger gradient when the discriminator is too good at recognizing fake samples.

$\frac{1}{m}\sum_{i=1}^{m}-\log(D(G(\boldsymbol{z}^{(i)})))$

Which is the Binary crossentropy between $D(G(\boldsymbol{z}^{(i)}))$ and the probability distribution that has $y^{(i)} = 1 \forall i$, i.e. we are forcing the generator to produce samples that will make the discriminator predict that fake samples are real.

Moreover we add a regularizer term so-called feature matching such that the distributions of the real and generated data are enforced to be close.

$\lambda_1 ||E_{x \sim p(x)} [x] - E_{z\sim p(z)} [G(z)] ||^2 + \lambda_2 ||E_{x \sim p(x)} [f(x)] - E_{z \sim p(z)} [f(G(z))] ||^2$


In [13]:
def generator_loss(fake_output, real_bar, fake_bar, real_f, fake_f, device, lambda1=0.01, lambda2=0.01):

    gen_loss = cross_entropy(fake_output, torch.ones_like(fake_output, device=device))

    mean_real = torch.mean(real_bar, dim=0, dtype=torch.float)
    mean_fake = torch.mean(fake_bar, dim=0, dtype=torch.float)
    l2_data = MSE(mean_real, mean_fake)

    mean_real_feat = torch.mean(real_f, dim=0, dtype=torch.float)
    mean_fake_feat = torch.mean(fake_f, dim=0, dtype=torch.float)
    l2_feat = MSE(mean_real_feat, mean_fake_feat)

    return gen_loss+lambda1*l2_data+lambda2*l2_feat

### Optimizers

With DCGAN the training is very diffuclt so we decide to use Adam optimizer as suggested by the paper. Note that with Adam we use both momentum and RMSprop to normalized velocities. Discriminator and generator need two different optimizers (conditioner is included in the generator training).

In [14]:
generator.to(device)
discriminator.to(device)

gen_opt = optim.Adam(generator.parameters(), lr=2e-4)
dis_opt = optim.Adam(discriminator.parameters(), lr=2e-4)

### Training loop

In [15]:
EPOCHS = 15
noise_dim = 100
BATCH_SIZE = 50


Data = PolyphonicDataset(Genre=['rock'], EightBars = True)
dataloader = DataLoader(Data, BATCH_SIZE, shuffle=True, num_workers=0, drop_last=True)

In [18]:
def train_step(bars, cond_1d, generator, discriminator, batch_size, noise_dim, device, dis_opt, gen_opt):
   noise = torch.randn([batch_size, noise_dim], device=device)

   # Generate Images
   generated_bars = generator(noise, cond_1d, batch_size)

   # Train Discriminator
   # Classify true and fake images (detach fake images for discriminator training)
   output, D, fm = discriminator(bars, cond_1d)
   fake_output, fake_D, fake_fm = discriminator(generated_bars.detach(), cond_1d)

   # Compute discriminator loss and update discriminator
   disc_loss = discriminator_loss(D, fake_D, device)
   discriminator.zero_grad()
   disc_loss.backward()
   dis_opt.step()

   # Train Generator
   # Get fresh feature maps for generator training (both real and fake)
   real_output, real_D, real_fm = discriminator(bars, cond_1d)
   fake_output, fake_D, fake_fm = discriminator(generated_bars, cond_1d)

   # Compute generator loss and update generator
   gen_loss = generator_loss(fake_D, bars, generated_bars, real_fm, fake_fm, device)
   generator.zero_grad()
   gen_loss.backward()
   gen_opt.step()

   return gen_loss, disc_loss

supponendo che nel dataloader ogni dato sia una bar + la preavious bar + 1d condition sugli strumenti utilizzati

In [19]:
gloss = []
dloss = []
cond_1d = torch.normal(0,1,[BATCH_SIZE,5])

for epoch in range(EPOCHS):

    start = time.time()
    generator.train()
    discriminator.train()
    gen_losses = []
    disc_losses = []
    print('#################')
    print(f"Epoch: {epoch+1}")

    iterator = tqdm(dataloader)
    for bar_batch in iterator:
        bar_batch = bar_batch.to(device)
        #prev_bar_batch = prev_bar_batch.to(device)
        instrument_batch = torch.zeros_like(cond_1d)
        instrument_batch = instrument_batch.to(device)

        gen_loss, disc_loss = train_step(bar_batch, instrument_batch, generator, discriminator, BATCH_SIZE, noise_dim, device, dis_opt, gen_opt)
        gen_losses.append(gen_loss.detach().cpu().numpy())
        disc_losses.append(disc_loss.detach().cpu().numpy())

        iterator.set_description('Discriminator loss: {}, Generator loss: {}'.format(disc_loss.detach().cpu().numpy(),gen_loss.detach().cpu().numpy()))

    gloss.append(np.mean(gen_losses))
    dloss.append(np.mean(disc_losses))
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))


#################
Epoch: 1


Discriminator loss: 0.3417219817638397, Generator loss: 6.015914440155029: 100%|██████████| 54/54 [00:11<00:00,  4.57it/s]  


Time for epoch 1 is 11.832403898239136 sec
#################
Epoch: 2


Discriminator loss: 0.3335947096347809, Generator loss: 6.328534126281738: 100%|██████████| 54/54 [00:09<00:00,  5.43it/s]  


Time for epoch 2 is 9.94576096534729 sec
#################
Epoch: 3


Discriminator loss: 0.3323092460632324, Generator loss: 6.547445774078369:  63%|██████▎   | 34/54 [00:06<00:03,  5.24it/s]  


KeyboardInterrupt: 

### Weights and loss analysis

First let's plot the 2 losses over the epochs, if it works correctly the generator loss would have to decrease and the discriminator one would have to increase

In [ ]:
plt.plot(gloss, label='Generator loss')
plt.plot(dloss, label='Discriminator loss')
plt.legend()
plt.show()

Then we can analyze the weights distribution

In [ ]:
# First hidden layer
h1_w = net.fc1.weight.data.cpu().numpy()
h1_b = net.fc1.bias.data.cpu().numpy()

# Second hidden layer
h2_w = net.fc2.weight.data.cpu().numpy()
h2_b = net.fc2.bias.data.cpu().numpy()

# Output layer
out_w = net.out.weight.data.cpu().numpy()
out_b = net.out.bias.data.cpu().numpy()

# Weights histogram
fig, axs = plt.subplots(3, 1, figsize=(12,8))
axs[0].hist(h1_w.flatten(), 50)
axs[0].set_title('First hidden layer weights')
axs[1].hist(h2_w.flatten(), 50)
axs[1].set_title('Second hidden layer weights')
axs[2].hist(out_w.flatten(), 50)
axs[2].set_title('Output layer weights')
[ax.grid() for ax in axs]
plt.tight_layout()
plt.show()

In [ ]:
# lo avevmo fatto nel lab2 forse adesso un po' complicato visto che come input abbiamo un vettore con 100 componenti

def get_activation(layer, input, output):
    global activation
    activation = torch.sigmoid(output)

### Register hook
hook_handle = generator.cnn1.register_forward_hook(get_activation)

### Analyze activations
generator = generator.to(device)
generator.eval()
with torch.no_grad():
    x1 = torch.tensor([0.1]).float().to(device)
    y1 = net(x1)
    z1 = activation
    x2 = torch.tensor([0.9]).float().to(device)
    y2 = net(x2)
    z2 = activation
    x3 = torch.tensor([2.5]).float().to(device)
    y3 = net(x3)
    z3 = activation

### Remove hook
hook_handle.remove()

### Plot activations
fig, axs = plt.subplots(3, 1, figsize=(12,6))
axs[0].stem(z1.cpu().numpy())
axs[0].set_title('Last layer activations for input x=%.2f' % x1)
axs[1].stem(z2.cpu().numpy())
axs[1].set_title('Last layer activations for input x=%.2f' % x2)
axs[2].stem(z3.cpu().numpy())
axs[2].set_title('Last layer activations for input x=%.2f' % x3)
plt.tight_layout()
plt.show()

#### Generator output

In [ ]:
generator.eval()
noise = torch.randn([1, noise_dim], device=device)
instrument =
prev_bar =


with torch.no_grad():
   generated_bar = generator(noise, prev_bar, instrument, 1)

generated_bar=generated_bar.cpu().numpy()

Save networks and optimizers states

In [ ]:
torch.save(discriminator.state_dict(), 'discriminator_parameters.torch')
torch.save(generator.state_dict(), 'generator_parameters.torch')
print('Saved Model')

torch.save(dis_opt.state_dict(), 'dis_opt_state.torch')
torch.save(gen_opt.state_dict(), 'gen_opt_state.torch')
print('Saved Optimizer')

Reload

In [ ]:
discriminator = Discriminator(input_size=100, cond_1d_size=15, instrument_size=1)
discriminator.apply(weights_init)
discriminator.to(device)
# Load the state dict previously saved
discriminator_state_dict = torch.load('discriminator_parameters.torch')
# Update the network parameters
discriminator.load_state_dict(discriminator_state_dict)

generator = Generator(input_size=100, cond_1d_size=15, instrument_size=1, n_hlayers=128)
generator.apply(weights_init)
generator.to(device)
# Load the state dict previously saved
generator_state_dict = torch.load('generator_parameters.torch')
# Update the network parameters
generator.load_state_dict(generator_state_dict)


dis_opt = optim.Adam(discriminator.parameters(), lr=2e-4)
# Load the state dict previously saved
dis_opt_state_dict = torch.load('dis_opt_state.torch')
# Update the network parameters
dis_opt.load_state_dict(dis_opt_state_dict)

gen_opt = optim.Adam(generator.parameters(), lr=2e-4)
# Load the state dict previously saved
gen_opt_state_dict = torch.load('gen_opt_state.torch')
# Update the network parameters
gen_opt.load_state_dict(gen_opt_state_dict)